In [ ]:
#Twitter API Json Files Get

In [ ]:
#Step 0: Libraries
import requests
import os
import json
import socket
import sys
import errno
import time
import sys
import pandas as pd
from io import StringIO
import numpy as np
from IPython.display import Javascript

In [ ]:
#Step 0.5: Initialize data and counter
data = []
counter = 0

In [ ]:
#Step 1: Bearer_Token
bearer_token = os.environ.get(xxxxxxxxxxxxxx)

In [ ]:
#Step 2: bearer_oauth function
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = fxxxxxxxxxxxxxxx
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

In [ ]:
#Step 3: Get the Rules Function
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [ ]:
#Step 4: Delete all the Rules Function, to clean the slate 
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))

In [ ]:
#Step 5: Add the New Rules Function
def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "\"military invasion\" OR \"military attack\" OR \"military clash\" OR \"military assault\" OR \"seize position\" lang: en", "tag": "Goldstein Negative"},
        {"value": "\"seize possessions\" OR \"nonmilitary destruction\" OR \"nonmilitary injury\" OR \"force mobilization\" OR \"force exercise\" lang: en", "tag": "Goldstein Negative"},
        {"value": "\"diplomatic recognition\" OR \"substantive agreement\" OR \"economic aid\" OR \"military assistance\" OR \"grant priviledge\" lang: en", "tag": "Goldstein Positive"},
        {"value": "\"suspend sanctions\" OR \"call truce\" OR \"material assistance\" OR \"endorse position\" OR \"verbal support\" lang: en", "tag": "Goldstein Positive"},
        {"value": "\"nuclear weapons\" OR \"nuclear weapon\" OR \"ballistic missile\" OR \"nuclear conflict\" OR \"nuclear attack\" lang: en", "tag": "Nuclear Threat"},
        {"value": "\"cyber attack\" OR \"cyber capabilities\" OR \"cyber defense\" OR \"cyber warfare\" OR \"cyber terrorism\" lang: en", "tag": "Cyber Warfare"},
        {"value": "\"war risk\" OR \"war fear\" OR \"military threat\" lang: en", "tag": "War Threats"},
        {"value": "\"oil crisis\" OR \"oil price\" OR \"petroleum exporting\" OR \"crude oil\" OR \"oil production\" lang: en", "tag": "Oil Supply Shock"},
        {"value": "\"China Sea\" OR \"China relations\" OR \"China trade\" OR \"Thucydides trap\" OR \"Taiwan sales\" lang: en", "tag": "US-China Relations"},
        {"value": "\"state terrorism\" OR \"counter terrorism\" OR \"terrorist attack\" OR \"political violence\" OR \"global terrorism\" lang: en", "tag": "Terrorism"},
        {"value": "\"geopolitical risk\" OR \"geopolitical concern\" OR \"geopolitical tension\" OR \"geopolitical uncertainty\" lang: en", "tag": "Geopolitical Risks"},
        {"value": "\"invasión militar\" OR \"ataque militar\" OR \"enfrentamiento militar\" OR \"asalto militar\" OR \"apoderamiento de la posición\" lang: es", "tag": "Goldstein Negative"},
        {"value": "\"apoderamiento de los bienes\" OR \"destrucción no militar\" OR \"daño no militar\" OR \"mobilización de fuerzas\" OR \"ejercicio de fuerzas\" lang: es", "tag": "Goldstein Negative"},
        {"value": "\"reconocimiento diplomático\" OR \"acuerdo sustancial\" OR \"ayuda económica\" OR \"asistencia militar\" OR \"otorgar privilegios\" lang: es", "tag": "Goldstein Positive"},
        {"value": "\"suspensión de sanciones\" OR \"tregua\" OR \"material de asistencia\" OR \"respaldo de posición\" OR \"apoyo verbal\" lang: es", "tag": "Goldstein Positive"},
        {"value": "\"armas nucleares\" OR \"arma nuclear\" OR \"misil balístico\" OR \"conflicto nuclear\" OR \"ataque nuclear\" lang: es", "tag": "Nuclear Threat"},
        {"value": "\"ciberataque\" OR \"capacidades cibernéticas\" OR \"defensa cibernética\" OR \"guerra cibernética\" OR \"ciberterrorismo\" lang: es", "tag": "Cyber Warfare"},
        {"value": "\"riesgo de guerra\" OR \"miedo a la guerra\" OR \"amenaza militar\" lang: es", "tag": "War Threats"},
        {"value": "\"crisis del petróleo\" OR \"precio del petróleo\" OR \"exportación del petróleo\" OR \"petróleo crudo\" OR \"producción del petróleo\" lang: es", "tag": "Oil Supply Shock"},
        {"value": "\"Mar de China\" OR \"relaciones de China\" OR \"comercio con China\" OR \"trampa de Tucídides\" OR \"ventas de Taiwan\" lang: es", "tag": "US-China Relations"},
        {"value": "\"estado terrorista\" OR \"lucha contra el terrorismo\" OR \"ataque terrorista\" OR \"violencia política\" OR \"terrorismo global\" lang: es", "tag": "Terrorism"},
        {"value": "\"riesgo geopolítico\" OR \"preocupación geopolítica\" OR \"tensión geopolítica\" OR \"incertidumbre geopolítica\" lang: es", "tag": "Geopolitical Risks"},
        {"value": "\"invasion militaire\" OR \"attaque militaire\" OR \"affrontement militaire\" OR \"assaut militaire\" OR \"saisir une position\" lang: fr", "tag": "Goldstein Negative"},
        {"value": "\"saisir des possessions\" OR \"destruction non militaire\" OR \"blessure non militaire\" OR \"mobilisation des forces\" OR \"exercice des forces\" lang: fr", "tag": "Goldstein Negative"},
        {"value": "\"reconnaissance diplomatique\" OR \"accord de fond\" OR \"aide économique\" OR \"aide militaire\" OR \"accorder un privilège\" lang: fr", "tag": "Goldstein Positive"},
        {"value": "\"suspendre les sanctions\" OR \"demander une trêve\" OR \"assistance matérielle\" OR \"soutenir une position\" OR \"soutien verbal\" lang: fr", "tag": "Goldstein Positive"},
        {"value": "\"armes nucléaires\" OR \"arme nucléaire\" OR \"missile balistique\" OR \"conflit nucléaire\" OR \"attaque nucléaire\" lang: fr", "tag": "Nuclear Threat"},
        {"value": "\"cyberattaque\" OR \"capacités cybernétiques\" OR \"cyberdéfense\" OR \"cyberguerre\" OR \"cyberterrorisme\" lang: fr", "tag": "Cyber Warfare"},
        {"value": "\"risque de guerre\" OR \"peur de la guerre\" OR \"menace militaire\" lang: fr", "tag": "War Threats"},
        {"value": "\"crise du pétrole\" OR \"prix du pétrole\" OR \"exportation de pétrole\" OR \"pétrole brut\" OR \"production de pétrole\" lang: fr", "tag": "Oil Supply Shock"},
        {"value": "\"Mer de Chine\" OR \"relations avec la Chine\" OR \"commerce Chinois\" OR \"piège de Thucydide\" OR \"ventes à Taiwan\" lang: fr", "tag": "US-China Relations"},
        {"value": "\"terrorisme d'État\" OR \"contre-terrorisme\" OR \"attentat terroriste\" OR \"violence politique\" OR \"terrorisme mondial\" lang: fr", "tag": "Terrorism"},
        {"value": "\"risque géopolitique\" OR \"préoccupation géopolitique\" OR \"tension géopolitique\" OR \"incertitude géopolitique\" lang: fr", "tag": "Geopolitical Risks"}, 
        {"value": "\"invasão militar\" OR \"ataque militar\" OR \"conflito militar\" OR \"investida militar\" OR \"tomar posição\" lang: pt", "tag": "Goldstein Negative"},
        {"value": "\"tomar os bens\" OR \"destruição não militar\" OR \"danos não militares\" OR \"mobilização da força\" OR \"exercício da força\" lang: pt", "tag": "Goldstein Negative"},
        {"value": "\"reconhecimento diplomático\" OR \"acordo substantivo\" OR \"ajuda econômica\" OR \"assistência militar\" OR \"conceder privilégios\" lang: pt", "tag": "Goldstein Positive"},
        {"value": "\"suspender sanções\" OR \"pedir trégua\" OR \"assistência material\" OR \"endossar posição\" OR \"apoio verbal\" lang: pt", "tag": "Goldstein Positive"},
        {"value": "\"armas nucleares\" OR \"arma nuclear\" OR \"míssil balístico\" OR \"conflito nuclear\" OR \"ataque nuclear\" lang: pt", "tag": "Nuclear Threat"},
        {"value": "\"ataque cibernético\" OR \"habilidades cibernéticas\" OR \"defesa cibernética\" OR \"guerra cibernética\" OR \"terrorismo cibernético\" lang: pt", "tag": "Cyber Warfare"},
        {"value": "\"risco de guerra\" OR \"medo de guerra\" OR \"ameaça militar\" lang: pt", "tag": "War Threats"},
        {"value": "\"crise do petróleo\" OR \"preço do petróleo\" OR \"exportação de petróleo\" OR \"petróleo bruto\" OR \"produção de petróleo\" lang: pt", "tag": "Oil Supply Shock"},
        {"value": "\"Mar da China\" OR \"relações com a China\" OR \"comércio da China\" OR \"armadilha de Tucídides\" OR \"vendas em Taiwan\" lang: pt", "tag": "US-China Relations"},
        {"value": "\"terrorismo de estado\" OR \"contraterrorismo\" OR \"ataque terrorista\" OR \"violência política\" OR \"terrorismo global\" lang: pt", "tag": "Terrorism"},
        {"value": "\"risco geopolítico\" OR \"preocupação geopolítica\" OR \"tensão geopolítica\" OR \"incerteza geopolítica\" lang: pt", "tag": "Geopolitical Risks"},
        {"value": "\"غزو عسكري\" OR \"هجوم عسكري\" OR \"صدام عسكري\" OR \"اعتداء عسكري\" OR \"تولي المنصب\" lang: ar", "tag": "Goldstein Negative"},
        {"value": "\"الاستيلاء على الممتلكات\" OR \"تدمير غير عسكري\" OR \"إصابة غير عسكرية\" OR \"حشد القوات\" OR \"تدريب القوات\" lang: ar", "tag": "Goldstein Negative"},
        {"value": "\"الاعتراف الدبلوماسي\" OR \"اتفاق جوهري\" OR \"عون اقتصادي\" OR \"مساعدة عسكرية\" OR \"منح الامتياز\" lang: ar", "tag": "Goldstein Positive"},
        {"value": "\"تعليق الجزاءات\" OR \"عقد هدنة\" OR \"المساعدة المادية\" OR \"تأييد الموقف\" OR \"دعم شفوي\" lang: ar", "tag": "Goldstein Positive"},
        {"value": "\"الأسلحة النووية\" OR \"سلاح نووي\" OR \"القذائف التسيارية\" OR \"راع نووي\" OR \"هجوم نووي\" lang: ar", "tag": "Nuclear Threat"},
        {"value": "\"الهجوم الإلكتروني\" OR \"القدرات الإلكترونية\" OR \"الدفاع الإلكتروني\" OR \"الحرب الإلكترونية\" OR \"إرهاب إلكتروني\" lang: ar", "tag": "Cyber Warfare"},
        {"value": "\"مخاطر الحرب\" OR \"مخاوف الحرب\" OR \"التهديد العسكر\" lang: ar", "tag": "War Threats"},
        {"value": "\"أزمة النفط\" OR \"سعر النفط\" OR \"تصدير البترول\" OR \"النفط الخام\" OR \"إنتاج النفط\" lang: ar", "tag": "Oil Supply Shock"},
        {"value": "\"بحر الصين\" OR \"العلاقات الصينية\" OR \"تجارة الصين\" OR \"مصيدة توسيديس\" OR \"مبيعات تايوان\" lang: ar", "tag": "US-China Relations"},
        {"value": "\"إرهاب الدولة\" OR \"مكافحة الإرهاب\" OR \"هجوم إرهابي\" OR \"عنف سياسي\" OR \"إرهاب عالمي\" lang: ar", "tag": "Terrorism"},
        {"value": "\"مخاطر جغرافية سياسية\" OR \"مخاوف جغرافية سياسية\" OR \"توترات جغرافية سياسية\" OR \"شكوك جغرافية سياسية\" lang: ar", "tag": "Geopolitical Risks"},
        {"value": "\"軍による侵攻\" OR \"軍による攻撃\" OR \"軍の衝突\" OR \"軍による暴行\" OR \"奪取地点\" lang: ja", "tag": "Goldstein Negative"},
        {"value": "\"所持品を奪取\" OR \"軍以外による破壊行為\" OR \"軍以外の負傷\" OR \"部隊の動員\" OR \"部隊の訓練\" lang: ja", "tag": "Goldstein Negative"},
        {"value": "\"外交的認識\" OR \"重要な合意\" OR \"経済支援\" OR \"軍の支援\" OR \"権限を付与\" lang: ja", "tag": "Goldstein Positive"},
        {"value": "\"制裁を一時中止\" OR \"休戦を呼びかける\" OR \"物的援助\" OR \"立場を支持\" OR \"言葉での支援\" lang: ja", "tag": "Goldstein Positive"},
        {"value": "\"核兵器\" OR \"核兵器\" OR \"弾道ミサイル\" OR \"核紛争\" OR \"核攻撃\" lang: ja", "tag": "Nuclear Threat"},
        {"value": "\"サイバー攻撃\" OR \"サイバー能力\" OR \"サイバー防衛\" OR \"サイバー空間での交戦\" OR \"サイバーテロ\" lang: ja", "tag": "Cyber Warfare"},
        {"value": "\"戦争のリスク\" OR \"戦争の恐怖\" OR \"軍の脅迫\" lang: ja", "tag": "War Threats"},
        {"value": "\"石油危機\" OR \"原油価格\" OR \"石油の輸出\" OR \"原油\" OR \"原油生産\" lang: ja", "tag": "Oil Supply Shock"},
        {"value": "\"シナ海\" OR \"中国との関係\" OR \"中国の貿易\" OR \"トゥキュディデスの罠\" OR \"台湾の売り上げ\" lang: ja", "tag": "US-China Relations"},
        {"value": "\"国家主導のテロ\" OR \"テロ対策\" OR \"テロ攻撃\" OR \"政治的暴力\" OR \"グローバルテロリズム\" lang: ja", "tag": "Terrorism"},
        {"value": "\"地政治学上のリスク\" OR \"地政治学上の懸念\" OR \"地政治学上の緊張\" OR \"地政治学上の不確定要素\" lang: ja", "tag": "Geopolitical Risks"},
        {"value": "\"군사적 침공\" OR \"군사적 공격\" OR \"군사적 충돌\" OR \"군사적 공격\" OR \"진지 점령\" lang: ko", "tag": "Goldstein Negative"},
        {"value": "\"소유물 탈취\" OR \"비군사적 파괴\" OR \"비군사적 부상\" OR \"무력 동원\" OR \"무력 훈련\" lang: ko", "tag": "Goldstein Negative"},
        {"value": "\"외교적 인정\" OR \"실질적 합의\" OR \"경제 원조\" OR \"군사 지원\" OR \"권한 부여\" lang: ko", "tag": "Goldstein Positive"},
        {"value": "\"제재 유예\" OR \"휴전 선언\" OR \"물적 지원\" OR \"입장 지지\" OR \"구두 지원\" lang: ko", "tag": "Goldstein Positive"},
        {"value": "\"핵무기\" OR \"핵무기\" OR \"탄도 미사일\" OR \"핵 분쟁\" OR \"핵공격\" lang: ko", "tag": "Nuclear Threat"},
        {"value": "\"사이버 공격\" OR \"사이버 능력\" OR \"사이버 방어\" OR \"사이버 워페이스\" OR \"사이버 테러\" lang: ko", "tag": "Cyber Warfare"},
        {"value": "\"전쟁 위험\" OR \"전쟁 공포\" OR \"군사적 위협\" lang: ko", "tag": "War Threats"},
        {"value": "\"석유 파동\" OR \"유가\" OR \"석유 수출\" OR \"원유\" OR \"석유 생산\" lang: ko", "tag": "Oil Supply Shock"},
        {"value": "\"중국해\" OR \"중국 관계\" OR \"중국 무역\" OR \"투키디데스 함정\" OR \"대만 판매\" lang: ko", "tag": "US-China Relations"},
        {"value": "\"국가 테러\" OR \"대테러\" OR \"테러 공격\" OR \"정치적 폭력\" OR \"글로벌 테러\" lang: ko", "tag": "Terrorism"},
        {"value": "\"지정학적 위험\" OR \"지정학적 관심사\" OR \"지정학적 긴장\" OR \"지정학적 불확실성\" lang: ko", "tag": "Geopolitical Risks"},       
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [ ]:
#Merge Dictionaries Function
#https://www.geeksforgeeks.org/python-merging-two-dictionaries/
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [ ]:
#Step 6: Get Stream API Started Function
def get_stream(set):
    global data, counter
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent = 4, sort_keys = True))
            data_dic = json_response['data']
            tag_str = json_response['matching_rules'][0]["tag"]
            tag_dict_temp = {"tag": tag_str}
            #tag_dic_temp = json.loads(tag_str_temp)
            #https://favtutor.com/blogs/string-to-dict-python
            full_Merge_dic = Merge(data_dic, tag_dict_temp)
            #data.append(json_response['data'])
            #data.append(json_response['matching_rules'][0]["tag"])
            #data_dic = json_response.get('data')
            #created_at_dic = data_dic.get('created_at')
            #text_dic = data_dic.get('text')
            #matching_dic = json_response.get('matching_rules')
            #tag_dic = matching_dic[0]
            #data_Merge_dic = Merge(created_at_dic, text_dic)
            #full_Merge_dic = Merge(data_dic, matching_dic)
            #data.append(data_dic)
            data.append(full_Merge_dic)
            #Store every 100 tweets
            #while counter < 3:
            if len(data) % 100 == 0:
                print('storing data')
                temp = json.dumps(data)
                pd.read_json(StringIO(temp)).to_json(f'/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/foo9/json_file/tw_example_{counter}.json', orient='records')
                data = []
                counter += 1

In [ ]:
#Main function 2 test
#https://docs.python.org/3/library/signal.html#note-on-sigpipe
def main_2(): 
        rules = get_rules()
        delete = delete_all_rules(rules)
        set = set_rules(delete)
        get_stream(set)

In [ ]:
#Define the second stream connection function
#Step 6: Get Stream API Started Function
def get_stream_2(set):
    global data, counter
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream?tweet.fields=created_at", auth=bearer_oauth, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent = 4, sort_keys = True))
            data_dic = json_response['data']
            tag_str = json_response['matching_rules'][0]["tag"]
            tag_dict_temp = {"tag": tag_str}
            #tag_dic_temp = json.loads(tag_str_temp)
            #https://favtutor.com/blogs/string-to-dict-python
            full_Merge_dic = Merge(data_dic, tag_dict_temp)
            #data.append(json_response['data'])
            #data.append(json_response['matching_rules'][0]["tag"])
            #data_dic = json_response.get('data')
            #created_at_dic = data_dic.get('created_at')
            #text_dic = data_dic.get('text')
            #matching_dic = json_response.get('matching_rules')
            #tag_dic = matching_dic[0]
            #data_Merge_dic = Merge(created_at_dic, text_dic)
            #full_Merge_dic = Merge(data_dic, matching_dic)
            #data.append(data_dic)
            data.append(full_Merge_dic)
            #Store every 100 tweets
            #while counter < 3:
            if len(data) % 100 == 0:
                print('storing data')
                temp = json.dumps(data)
                pd.read_json(StringIO(temp)).to_json(f'/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/foo9/json_file/tw_example_{counter}_2.json', orient='records')
                data = []
                counter += 1

In [ ]:
#Main function 2 test
#https://docs.python.org/3/library/signal.html#note-on-sigpipe
def main_2_two(): 
        rules = get_rules()
        delete = delete_all_rules(rules)
        set = set_rules(delete)
        get_stream_2(set)

In [ ]:
#Step 9: Connect the Local Host to Spark for the Twitter API to send Tweets to Spark
#if __name__ == "__main__":
#    main_2()


In [ ]:
#Step 9 Alt: Reconnection automatically
#https://stackoverflow.com/questions/32267540/jupyter-notebook-how-to-relaunch-all-cells-above-when-a-crash-occurs
if __name__ == "__main__":
    try:
        main_2()
    except:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'+
                           ' IPython.notebook.get_selected_index()+2)'))

In [ ]:
#Step 9 Alt: Reconnection automatically
#https://stackoverflow.com/questions/32267540/jupyter-notebook-how-to-relaunch-all-cells-above-when-a-crash-occurs
if __name__ == "__main__":
    try:
        main_2_two()
    except:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'+
                           ' IPython.notebook.get_selected_index()+2)'))

In [ ]:
#Step 9 Alt: Reconnection automatically
#https://stackoverflow.com/questions/32267540/jupyter-notebook-how-to-relaunch-all-cells-above-when-a-crash-occurs
if __name__ == "__main__":
    try:
        main_2_two()
    except:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'+
                           ' IPython.notebook.get_selected_index()+2)'))

In [ ]:
#Step 9 Alt: Reconnection automatically
#https://stackoverflow.com/questions/32267540/jupyter-notebook-how-to-relaunch-all-cells-above-when-a-crash-occurs
if __name__ == "__main__":
    try:
        main_2_two()
    except:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1,'+
                           ' IPython.notebook.get_selected_index()+2)'))

In [ ]:
print("end")
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,'+
#                           ' IPython.notebook.get_selected_index()-1)'))

In [ ]:
#print("1")

In [ ]:
#print("2")

In [ ]:
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index(),'+ 
#                   ' IPython.notebook.get_selected_index()+1)'))

In [ ]:
#print("5")

In [ ]:
#print("4")

In [ ]:
#print("1")

In [ ]:
#print("2")

In [ ]:
#display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()-2,'+
#                           ' IPython.notebook.get_selected_index()-1)'))

In [ ]:
#print("3")

In [ ]:
#print("4")